# Part 3 – Multi-Agent Mayhem: The Router Agent 🚏

In this section, we introduce the **Router Agent**, a top-level controller whose only job is to analyze a user query and decide **which specialized agent** should handle it.

---

## Concept: Why Router Agents? 🤔
- A single agent has limited abilities.  
- Complex user queries often span multiple domains (travel, food, transportation, events).  
- The Router Agent acts as a **dispatcher**:
  - It does **not** answer the query itself.  
  - It **returns the name** of the best specialized agent for the job.  

---




Part 0: Setup & Authentication 🔑

In [8]:
!pip install google-adk google-generativeai -q

# --- Import all necessary libraries for our entire adventure ---
import os
import re
import asyncio
from IPython.display import display, Markdown
import google.generativeai as genai
from google.adk.agents import Agent, SequentialAgent, LoopAgent, ParallelAgent
from google.adk.tools import google_search, ToolContext
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from getpass import getpass

print("✅ All libraries are ready to go!")

✅ All libraries are ready to go!


In [9]:
# --- Securely Configure Your API Key ---

# Prompt the user for their API key securely
api_key = getpass('Enter your Google API Key: ')

# Get Your API Key HERE 👉 https://codelabs.developers.google.com/onramp/instructions#0
# Configure the generative AI library with the provided key
genai.configure(api_key=api_key)

# Set the API key as an environment variable for ADK to use
os.environ['GOOGLE_API_KEY'] = api_key

print("✅ API Key configured successfully! Let the fun begin.")

✅ API Key configured successfully! Let the fun begin.


In [10]:
# --- A Helper Function to Run Our Agents ---
# We'll use this function throughout the notebook to make running queries easy.

async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str, is_router: bool = False):
    """Initializes a runner and executes a query for a given agent and session."""
    print(f"\n🚀 Running query for agent: '{agent.name}' in session: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if not is_router:
                # Let's see what the agent is thinking!
                print(f"EVENT: {event}")
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"An error occurred: {e}"

    if not is_router:
     print("\n" + "-"*50)
     print("✅ Final Response:")
     display(Markdown(final_response))
     print("-"*50 + "\n")

    return final_response

# --- Initialize our Session Service ---
# This one service will manage all the different sessions in our notebook.
session_service = InMemorySessionService()
my_user_id = "adk_adventurer_001"


## Step 1 – Define Specialized Agents 🧑‍💻

In [11]:
#-- Agent Definitions for our Specialist Team --

day_trip_agent = Agent(
    name="day_trip_agent",
    model="gemini-2.5-flash",
    description="Agent specialized in generating spontaneous full-day itineraries based on mood, interests, and budget.",
    instruction="""
    You are the "Spontaneous Day Trip" Generator 🚗 - a specialized AI assistant that creates engaging full-day itineraries.

    Your Mission:
    Transform a simple mood or interest into a complete day-trip adventure with real-time details, while respecting a budget.

    Guidelines:
    1. **Budget-Aware**: Pay close attention to budget hints like 'cheap', 'affordable', or 'splurge'. Use Google Search to find activities (free museums, parks, paid attractions) that match the user's budget.
    2. **Full-Day Structure**: Create morning, afternoon, and evening activities.
    3. **Real-Time Focus**: Search for current operating hours and special events.
    4. **Mood Matching**: Align suggestions with the requested mood (adventurous, relaxing, artsy, etc.).

    RETURN itinerary in MARKDOWN FORMAT with clear time blocks and specific venue names.
    """,
    tools=[google_search]

)

foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are an expert food critical. Your goal is to find the absolute best food, restaurant, or culinary experiences basedAgent" \
    "on a user's request. When you recommend a place, state its name clearly. For example: 'The best sushi is at **Jin Sho**. "
)

weekend_guide_agent=Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

transportation_agent=Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are an navigatoin assistant. Given a starting point and destination, provide clear directions on how to get from the start to the end."
)


# Step 2 – Define the Router Agent 🚏

In [12]:
# --- The Brain of the Operation: The Router Agent ---
# We update the router's instructions to know about the new 'combo' task.

router_agent=Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
        You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_combo': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We'll create a dictionary of all our individual worker agents
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "transportation_agent": transportation_agent, # Add the new agent!
}

print("🤖 Agent team assembled for sequential workflows!")

🤖 Agent team assembled for sequential workflows!


# Step 3 – Sequential Workflow (Router → Specialist) 🔀

In [13]:
# --- Let's Test the Sequential Workflow! ---

async def run_sequential_app():
    queries = [
        "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the COMBO
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        #1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent,query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        if chosen_route == 'find_and_navigate_combo':
            print("\n--- Starting Find and Navigate Combo Workflow ---")

            # STEP 2a: Run the foodie_agent first
            foodie_session = await session_service.create_session(app_name=foodie_agent.name, user_id=my_user_id)
            foodie_response = await run_agent_query(foodie_agent, query, foodie_session, my_user_id)

            # Step 2b: Extract the destination from the first agent's response
            #(This is a simple regex, a more robust solution might use a structured output format)
            match = re.search(r'\*\*(.*?)\*\*', foodie_response)
            if not match:
                print("🚨 Could not determine the restaurant name from the response.")
                continue
            destination = match.group(1)
            print(f" Extract Destination: {destination}")

            #Step 2c: Create a new query and run the transportation_agent
            directions_query = f"Give me directions to {destination} from the Palo Alto Caltrain station."
            print(f"\n🗣️ New Query for Transport Agent: '{directions_query}'")
            transport_session = await session_service.create_session(app_name=transportation_agent.name, user_id=my_user_id)
            await run_agent_query(transportation_agent, directions_query, transport_session, my_user_id)

            print("--- Combo Workflow Complete ---")

        elif chosen_route in worker_agents:
            #This is a simple, single-agent route
            worker_agent = worker_agents[chosen_route]
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session,my_user_id)
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")    

await run_sequential_app()




🗣️ Processing New Query: 'I want to eat the best sushi in Palo Alto.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'f383ae7c-e8a1-4bc7-abbf-b7315b0bd517'...
🚦 Router has selected route: 'foodie_agent'

🚀 Running query for agent: 'foodie_agent' in session: 'e74317cb-97c9-45b5-af15-2d7bf1de5659'...
EVENT: content=Content(
  parts=[
    Part(
      text="""For the best sushi experience in Palo Alto, **Jin Sho** is a consistently highly recommended choice. It's known for its exceptionally fresh fish, including buttery otoro and creamy uni, and has even been noted as a favorite of Steve Jobs. Diners describe it as a "culinary haven" with a chic setting, offering expertly crafted sushi and cooked dishes like black cod with miso.

Other excellent options for sushi in Palo Alto include:

*   **Fuki Sushi**: This traditional Japanese restaurant is praised for its classic dishes, captivating interior design, and delicately crafted sushi 

For the best sushi experience in Palo Alto, **Jin Sho** is a consistently highly recommended choice. It's known for its exceptionally fresh fish, including buttery otoro and creamy uni, and has even been noted as a favorite of Steve Jobs. Diners describe it as a "culinary haven" with a chic setting, offering expertly crafted sushi and cooked dishes like black cod with miso.

Other excellent options for sushi in Palo Alto include:

*   **Fuki Sushi**: This traditional Japanese restaurant is praised for its classic dishes, captivating interior design, and delicately crafted sushi rolls with fresh seafood.
*   **Sushi Tomi**: A cozy spot, Sushi Tomi is recognized for its fresh fish, traditional sushi offerings, and an authentic atmosphere. It's known to have chef's choice meals and a variety of Japanese delicacies.
*   **Araki Sushi**: Located in the heart of Palo Alto, Araki Sushi offers authentic Japanese dining with masterful sushi craftsmanship using California's freshest ingredients.

--------------------------------------------------


🗣️ Processing New Query: 'Are there any cool outdoor concerts this weekend?'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '420449aa-8b68-4785-ab14-8127c23a44f5'...
🚦 Router has selected route: 'weekend_guide_agent'

🚀 Running query for agent: 'weekend_guide_agent' in session: '483d76d4-7e87-4714-aa1a-506ead7a47b7'...


CancelledError: 

# Part 2 (The ADK Way): Multi-Agent Mayhem with SequentialAgent 🧠→⛓️→🤖

You've seen how to manually link agents together with custom Python code. It works, but it can get complicated. Now, let's refactor our workflow to use a powerful, built-in ADK feature designed specifically for this: the SequentialAgent.

The SequentialAgent is a workflow agent. It's not powered by an LLM itself; instead, its only job is to execute a list of other agents in a strict, predefined order.

The real magic ✨ is how it passes information. The ADK uses a shared state dictionary that each agent in the sequence can read from and write to.

Our New Workflow:

Foodie Agent: Finds the restaurant and saves the name to state['destination'].
Transportation Agent: Automatically reads state['destination'] and uses it to find directions.
This means we no longer need custom Python code to extract text or build new queries! The ADK handles the plumbing for us.

In [14]:
# --- Agent Definitions for our Specialist Team (Refactored for Sequential Workflow) ---

# ✨ CHANGE 1: We tell foodie_agent to save its output to the shared state.
# Note the new `output_key` and the more specific instruction.
foodie_agent = Agent(
    name="foodie_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment and nothing else.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="destination"  # ADK will save the agent's final response to state['destination']
)

# ✨ CHANGE 2: We tell transportation_agent to read from the shared state.
# The `{destination}` placeholder is automatically filled by the ADK from the state.
transportation_agent = Agent(
    name="transportation_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are a navigation assistant. Given a destination, provide clear directions.
    The user wants to go to: {destination}.

    Analyze the user's full original query to find their starting point.
    Then, provide clear directions from that starting point to {destination}.
    """,
)

# ✨ CHANGE 3: Define the SequentialAgent to manage the workflow.
# This agent will run foodie_agent, then transportation_agent, in that exact order.
find_and_navigate_agent = SequentialAgent(
    name="find_and_navigate_agent",
    sub_agents=[foodie_agent, transportation_agent],
    description="A workflow that first finds a location and then provides directions to it."
)

weekend_guide_agent = Agent(
    name="weekend_guide_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="You are a local events guide. Your task is to find interesting events, concerts, festivals, and activities happening on a specific weekend."
)

# --- The Brain of the Operation: The Router Agent ---

# We update the router to know about our new, powerful SequentialAgent.
router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about food, restaurants, or eating.
    - 'weekend_guide_agent': For queries about events, concerts, or activities happening on a specific timeframe like a weekend.
    - 'day_trip_agent': A general planner for any other day trip requests.
    - 'find_and_navigate_agent': Use this for complex queries that ask to *first find a place* and *then get directions* to it.

    Only return the single, most appropriate option's name and nothing else.
    """
)

# We create a dictionary of all our executable agents for easy lookup.
# This now includes our powerful new workflow agent!
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent,
    "weekend_guide_agent": weekend_guide_agent,
    "find_and_navigate_agent": find_and_navigate_agent, # Add the new sequential agent
}

print("🤖 Agent team assembled with a SequentialAgent workflow!")




🤖 Agent team assembled with a SequentialAgent workflow!


In [15]:
# --- Let's Test the Streamlined Workflow! ---

async def run_sequential_app():
    queries = [
         "I want to eat the best sushi in Palo Alto.", # Should go to foodie_agent
        "Are there any cool outdoor concerts this weekend?", # Should go to weekend_guide_agent
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station." # Should trigger the SequentialAgent
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflow
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

        # 2. Execute the chosen route
        # This logic is now much simpler! The SequentialAgent is treated just like any other worker.
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")
            
await run_sequential_app()






🗣️ Processing New Query: 'I want to eat the best sushi in Palo Alto.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'b209e19e-b255-4225-8815-fa0e70a55ecf'...
🚦 Router has selected route: 'foodie_agent'
--- Handing off to foodie_agent ---

🚀 Running query for agent: 'foodie_agent' in session: '725cd51d-442c-45e4-9e9f-6a8fe4ebde13'...
EVENT: content=Content(
  parts=[
    Part(
      text='Jin Sho'
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  search_entry_point=SearchEntryPoint(
    rendered_content="""<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line-height: 20px;
  padding: 8px 12px;
}
.chip {
  display: inline-block;
  border: solid 1px;
  border-radius: 16px;
  min-width: 14px;
  padding: 5px 16px;
  text-align: center;
  user-select: none;
  margin: 0 8px;
  -webkit-tap-highlight-color: transparen

Jin Sho

--------------------------------------------------

--- foodie_agent Complete ---

🗣️ Processing New Query: 'Are there any cool outdoor concerts this weekend?'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '7e5c8528-4523-4425-b935-c7e3f25f1064'...
🚦 Router has selected route: 'weekend_guide_agent'
--- Handing off to weekend_guide_agent ---

🚀 Running query for agent: 'weekend_guide_agent' in session: 'c8d5c11c-555f-467a-b923-88c763afaa6e'...
EVENT: content=Content(
  parts=[
    Part(
      text="""This weekend, from September 6th to 8th, 2025, there are several exciting outdoor concerts and festivals happening across different locations:

**East Coast**

*   **Raleigh, North Carolina:** The **Hopscotch Music Festival** concludes this weekend, running through Saturday, September 6th. This festival in downtown Raleigh features outdoor stages and late-night shows with a lineup including Earl Sweatshirt, My Morning Jacket, Sparks, a

This weekend, from September 6th to 8th, 2025, there are several exciting outdoor concerts and festivals happening across different locations:

**East Coast**

*   **Raleigh, North Carolina:** The **Hopscotch Music Festival** concludes this weekend, running through Saturday, September 6th. This festival in downtown Raleigh features outdoor stages and late-night shows with a lineup including Earl Sweatshirt, My Morning Jacket, Sparks, and Built to Spill.
*   **Hershey, Pennsylvania:** Hersheypark Stadium will host two concerts. **HARDY: Jim Bob World Tour** is scheduled for Saturday, September 6th, at 6:15 PM, and **Falling In Reverse** will perform on Sunday, September 7th, at 6:15 PM.
*   **New York City, New York:** Numerous free outdoor concerts are available through NYC Parks:
    *   On Saturday, September 6th, "The Zardon and Friends of Brooklyn" Showcase will take place at the Herbert Von King Cultural Arts Center in Brooklyn from 12:00 PM to 5:30 PM. Also on Saturday, the Harlem Chamber Players Concert will be held at the Harlem Rose Garden in Manhattan from 12:00 PM to 1:00 PM, celebrating Latinx Heritage Month.
    *   Bryant Park will host "Picnic Performances: New York City Opera: Carmen" on both Friday, September 5th, and Saturday, September 6th, from 7:00 PM to 9:00 PM.
    *   On Sunday, September 7th, the "Summer on the Hudson: West Side County Fair" at Pier I in Riverside Park South, Manhattan, will offer live bands, carnival rides, and more from 1:00 PM to 6:00 PM.

**West Coast**

*   **Denver, Colorado:** The renowned Red Rocks Amphitheatre will feature **Brandi Carlile** from Friday, September 5th, through Sunday, September 7th. On Monday, September 8th, **Rainbow Kitten Surprise with Medium Build** will perform.
*   **Lake Tahoe, California:** The Lake Tahoe Summer Concert Series at the Lake Tahoe Amphitheatre at Caesars Republic will present **Teddy Swims** on Saturday, September 6th.
*   **Near Portland, Oregon:** Edgefield Concerts in Troutdale will host **Gregory Alan Isakov with Dean Johnson** on Sunday, September 7th.

--------------------------------------------------

--- weekend_guide_agent Complete ---

🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '99537a68-f0cf-41c7-aae7-c1a27219059c'...
🚦 Router has selected route: 'find_and_navigate_agent'
--- Handing off to find_and_navigate_agent ---

🚀 Running query for agent: 'find_and_navigate_agent' in session: 'ee528f7c-25ff-4f90-a608-1a6db3d569f0'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Jin Sho

To get to Jin Sho from the Caltrain station, go to the California Avenue Caltrain station. Jin Sho is located at 454 California Avenue, Palo Alto, CA 94306. The California Avenue Caltrain station is very close to the restaurant, with "California Ave Caltrain (N) is 91 yards away, 2 min walk.""""
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(


To get to Jin Sho from the Caltrain station, head to the California Avenue Caltrain station. Jin Sho is located at 454 California Avenue, Palo Alto, CA 94306. From the California Avenue Caltrain station, Jin Sho is just a short walk of 91 yards, which takes approximately 2 minutes.

--------------------------------------------------

--- find_and_navigate_agent Complete ---


---
## Iterative Ideas with `LoopAgent` 🧠→🔁→🤖

Sometimes a task isn't a straight line; it's a loop of refinement. A user might ask for a plan, but have constraints that require checking and re-planning. For this, the ADK provides the **`LoopAgent`**.

The `LoopAgent` executes a sequence of sub-agents repeatedly until a condition is met. This is perfect for workflows involving trial and error, like planning a trip with a tight schedule.

**Our New Workflow: The Perfectionist Planner**

1. **Planner Agent**: Proposes an itinerary (e.g., a museum and a restaurant).
2. **Critic Agent**: Checks the plan against a constraint (e.g., "Is the travel time between these two places less than 30 minutes?").
3. **Refiner Agent**: If the critic finds a problem, this agent takes the feedback and creates a new, improved plan. If the critic is happy, it calls a special `exit_loop` tool to stop the process.

The `LoopAgent` manages this cycle, ensuring we don't get stuck in an infinite loop by setting a `max_iterations` limit.

In [16]:
# --- Agent Definitions for an Iterative Workflow ---

# A tool to signal that the loop should terminate
COMPLETION_PHRASE = "The plan is feasible and meets all constraints."
def exit_loop(tool_context: ToolContext):
  """Call this function ONLY when the plan is approved, signaling the loop should end."""
  print(f"  [Tool Call] exit_loop triggered by {tool_context.agent_name}")
  tool_context.actions.escalate = True
  return {}

#Agent 1: Propose a initial plan
planner_agent= Agent(
  name="planer_agent", model="gemini-2.5-flash", tools=[google_search],
  instruction="You are a trip planner. Based on the user's request, propose a single activity and a single restaurant. Output only the names, like: 'Activity: Exploratorium, Restaurant: La Mar'.",
  output_key="current_plan"
)

#Agent 2 (in loop): Critiques the plan
critic_agent = Agent(
  name="critic_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction=f"""You are a logistics expert. Your job is to critique a travel plan. The user has a strict constraint: total travel time must be short.
    Current Plan: {{current_plan}}
    Use your tools to check the travel time between the two locations.
    IF the travel time is over 45 minutes, provide a critique, like: 'This plan is inefficient. Find a restaurant closer to the activity.'
    ELSE, respond with the exact phrase: '{COMPLETION_PHRASE}'""",
    output_key="criticism"
)

#Agent 3 (in loop): Refines the plan or exits
refiner_agent = Agent(
    name="refiner_agent", model="gemini-2.5-flash", tools=[google_search, exit_loop],
    instruction=f"""You are a trip planner, refining a plan based on criticism.
    Original Request: {{session.query}}
    Critique: {{criticism}}
    IF the critique is '{COMPLETION_PHRASE}', you MUST call the 'exit_loop' tool.
    ELSE, generate a NEW plan that addresses the critique. Output only the new plan names, like: 'Activity: de Young Museum, Restaurant: Nopa'.""",
    output_key="current_plan"
)


# ✨ The LoopAgent orchestrates the critique-refine cycle ✨
refinement_loop = LoopAgent(
    name="refinement_loop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=3
)

# ✨ The SequentialAgent puts it all together ✨
iterative_planner_agent = SequentialAgent(
  name="iterative_planner_agent",
  sub_agents=[planner_agent, refinement_loop],
  description="A workflow that iteratively plans and refines a trip to meet constrains."
)

print("🤖 Agent team updated with an iterative LoopAgent workflow!")



🤖 Agent team updated with an iterative LoopAgent workflow!


---
## Parallel Power with `ParallelAgent` 🧠→⚡️→🤖🤖🤖

What if a user wants to find multiple, unrelated things at once? "Find me a museum, a concert, AND a restaurant." Running these searches one by one is slow and inefficient.

Enter the **`ParallelAgent`**. This workflow agent executes a list of sub-agents *concurrently*, dramatically speeding up tasks that can be performed independently.

**Our New Workflow: The Multi-Researcher**

1.  **Parallel Agent**: Simultaneously runs three specialist agents:
    - `MuseumFinderAgent`: Finds a museum.
    - `ConcertFinderAgent`: Finds a concert.
    - `FoodieAgent`: Finds a restaurant.
2.  **Synthesis Agent**: Once all three parallel searches are complete, this final agent gathers the results (which were saved to the shared `state`) and formats them into a single, neat summary for the user.

This pattern lets us get a lot of work done, fast! 🚀

In [17]:
# --- Agent Definitions for a Parallel Workflow ---

#Specialist Agent 1
museum_finder_agent = Agent(
    name="museum_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are a museum expert. Find the best museum based on the user's query. Output only the museum's name.",
    output_key="museum_result"
)

# Specialist Agent 2
concert_finder_agent = Agent(
    name="concert_finder_agent", model="gemini-2.5-flash", tools=[google_search],
    instruction="You are an events guide. Find a concert based on the user's query. Output only the concert name and artist.",
    output_key="concert_result"
)

# We can reuse our foodie_agent for the third parallel task!
# Just need to give it a new output_key for this workflow.
# restaurant_finder_agent = foodie_agent.copy(update={"output_key": "restaurant_result"})
restaurant_finder_agent = Agent(
    name="restaurant_finder_agent",
    model="gemini-2.5-flash",
    tools=[google_search],
    instruction="""You are an expert food critic. Your goal is to find the best restaurant based on a user's request.

    When you recommend a place, you must output *only* the name of the establishment.
    For example, if the best sushi is at 'Jin Sho', you should output only: Jin Sho
    """,
    output_key="restaurant_result" # Set the correct output key for this workflow
)

# ✨ The ParallelAgent runs all three specialists at once ✨
parallel_research_agent = ParallelAgent(
    name="parallel_research_agent",
    sub_agents=[museum_finder_agent, concert_finder_agent, restaurant_finder_agent]
)

# Agent to synthesize the parallel results
synthesis_agent = Agent(
    name="synthesis_agent", model="gemini-2.5-flash",
    instruction="""You are a helpful assistant. Combine the following research results into a clear, bulleted list for the user.
    - Museum: {museum_result}
    - Concert: {concert_result}
    - Restaurant: {restaurant_result}
    """
)

# ✨ The SequentialAgent runs the parallel search, then the synthesis ✨
parallel_planner_agent = SequentialAgent(
    name="parallel_planner_agent",
    sub_agents=[parallel_research_agent, synthesis_agent],
    description="A workflow that finds multiple things in parallel and then summarizes the results."
)

print("🤖 Agent team supercharged with a ParallelAgent workflow!")



🤖 Agent team supercharged with a ParallelAgent workflow!


---
### Final Step: Updating the Router and Running the App

Now we just have one last thing to do: make our `router_agent` aware of these powerful new workflows! We'll add `iterative_planner_agent` and `parallel_planner_agent` to its list of available options.

Then we can run our app with new queries designed to trigger these advanced, multi-agent workflows.

```
                    +---------------------+
                    |    User Query 🗣️     |
                    +----------+----------+
                               |
                               v
                    +---------------------+
                    |   Router Agent 🤖    |
                    |  (Classify Request) |
                    +----------+----------+
                               |
      +-----------+-----------+-----------+-----------+------------+
      |           |           |           |           |            |
      v           v           v           v           v            v
+-------------+  +------------------+  +------------------+  +------------------+  +-----------------+
| foodie_agent|  | find_and_navigate|  | iterative_planner|  | parallel_planner |  | day_trip_agent  |
| 🍣 Food Only |  | 🧭 Seq Workflow   |  | 🔁 Loop Workflow  |  | ⚡ Parallel Tasks |  | 🧳 Basic Plan     |
+-------------+  +------------------+  +------------------+  +------------------+  +-----------------+
```

In [18]:
# --- The ULTIMATE Router Agent --- #

router_agent = Agent(
    name="router_agent",
    model="gemini-2.5-flash",
    instruction="""
    You are a master request router. Your job is to analyze a user's query and decide which of the following agents or workflows is best suited to handle it.
    Do not answer the query yourself, only return the name of the most appropriate choice.

    Available Options:
    - 'foodie_agent': For queries *only* about finding a single food place.
    - 'find_and_navigate_agent': For queries that ask to *first find a place* and *then get directions* to it.
    - 'iterative_planner_agent': For planning a trip with a specific constraint that needs checking, like travel time.
    - 'parallel_planner_agent': For queries that ask to find multiple, independent things at once (e.g., a museum AND a concert AND a restaurant).
    - 'day_trip_agent': A general planner for any other simple day trip requests.

    Only return the single, most appropriate option's name and nothing else.
    """
)


# The master dictionary of all our executable agents and workflows
worker_agents = {
    "day_trip_agent": day_trip_agent,
    "foodie_agent": foodie_agent, # For simple food queries
    "find_and_navigate_agent": find_and_navigate_agent, # Sequential
    "iterative_planner_agent": iterative_planner_agent, # Loop
    "parallel_planner_agent": parallel_planner_agent,   # Parallel
}

# --- Let's Test Everything! ---
async def run_fully_loaded_app():
    queries = [
        # Test Case 1: Simple Sequential Flow
        "Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.",

        # Test Case 2: Iterative Loop Flow
        "Plan me a day in San Francisco with a museum and a nice dinner, but make sure the travel time between them is very short.",

        # Test Case 3: Parallel Flow
        "Help me plan a trip to SF. I need one museum, one concert, and one great restaurant."
    ]

    for query in queries:
        print(f"\n{'='*60}\n🗣️ Processing New Query: '{query}'\n{'='*60}")

        # 1. Ask the Router Agent to choose the right agent or workflowAgent
        router_session = await session_service.create_session(app_name=router_agent.name, user_id=my_user_id)
        print("🧠 Asking the router agent to make a decision...")
        chosen_route = await run_agent_query(router_agent, query, router_session, my_user_id, is_router=True)
        chosen_route = chosen_route.strip().replace("'", "")
        print(f"🚦 Router has selected route: '{chosen_route}'")

         # 2. Execute the chosen route
        if chosen_route in worker_agents:
            worker_agent = worker_agents[chosen_route]
            print(f"--- Handing off to {worker_agent.name} ---")
            worker_session = await session_service.create_session(app_name=worker_agent.name, user_id=my_user_id)
            await run_agent_query(worker_agent, query, worker_session, my_user_id)
            print(f"--- {worker_agent.name} Complete ---")
        else:
            print(f"🚨 Error: Router chose an unknown route: '{chosen_route}'")

await run_fully_loaded_app()






🗣️ Processing New Query: 'Find me the best sushi in Palo Alto and then tell me how to get there from the Caltrain station.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'dc19d2fd-88dd-4df9-b5d5-c09f2257b424'...
🚦 Router has selected route: 'find_and_navigate_agent'
--- Handing off to find_and_navigate_agent ---

🚀 Running query for agent: 'find_and_navigate_agent' in session: 'c7d2a7dc-63a4-4733-8994-7d88c276522f'...
EVENT: content=Content(
  parts=[
    Part(
      text="""Jin Sho

To get to Jin Sho from the California Avenue Caltrain Station, you can walk. Jin Sho is located at 454 California Avenue, Palo Alto, which is very close to the California Avenue Caltrain Station. The station itself is on California Avenue.

Walking directions:
1. Exit the California Avenue Caltrain Station.
2. Head towards 454 California Avenue. Jin Sho is just a short walk away on California Avenue."""
    ),
  ],
  role='model'
) grounding_metadat

If you're looking for excellent sushi in Palo Alto, Jin Sho comes highly recommended and is often cited as one of the best. It was even reportedly a favorite of Steve Jobs. Reviewers frequently praise the freshness of its fish, with the otoro and uni often highlighted as exceptional. Jin Sho is known for its yellowtail jalapeno sashimi and spicy tuna rolls, as well as its miso black cod.

You can find Jin Sho at 454 California Avenue, Palo Alto. It's conveniently located very close to the California Avenue Caltrain Station.

**Walking directions from the California Avenue Caltrain Station to Jin Sho:**
1.  Exit the California Avenue Caltrain Station.
2.  Head towards 454 California Avenue. Jin Sho is just a short walk away on California Avenue.

--------------------------------------------------

--- find_and_navigate_agent Complete ---

🗣️ Processing New Query: 'Plan me a day in San Francisco with a museum and a nice dinner, but make sure the travel time between them is very short.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: 'ece34823-3b18-45b7-8b4a-a872e28e446f'...
🚦 Router has selected route: 'iterative_planner_agent'
--- Handing off to iterative_planner_agent ---

🚀 Running query for agent: 'iterative_planner_agent' in session: '58e9a939-8dc1-4c6f-ba73-a2144a839168'...
EVENT: content=Content(
  parts=[
    Part(
      text='Activity: Exploratorium, Restaurant: La Mar Cocina Peruana'
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  search_entry_point=SearchEntryPoint(
    rendered_content="""<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google Sans, Roboto, sans-serif;
  font-size: 14px;
  line

An error occurred: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Tool use with function calling is unsupported', 'status': 'INVALID_ARGUMENT'}}

--------------------------------------------------

--- iterative_planner_agent Complete ---

🗣️ Processing New Query: 'Help me plan a trip to SF. I need one museum, one concert, and one great restaurant.'
🧠 Asking the router agent to make a decision...

🚀 Running query for agent: 'router_agent' in session: '47c11e65-7931-4fa7-9f1a-911cd42f1028'...
🚦 Router has selected route: 'parallel_planner_agent'
--- Handing off to parallel_planner_agent ---

🚀 Running query for agent: 'parallel_planner_agent' in session: 'f1ca2a14-62fc-43f9-9ddc-0bacabf22729'...
EVENT: content=Content(
  parts=[
    Part(
      text='The San Francisco Museum of Modern Art (SFMOMA) is an excellent choice for a museum.'
    ),
    Part(
      text='San Francisco Museum of Modern Art'
    ),
  ],
  role='model'
) grounding_metadata=GroundingMetadata(
  search_entry_point=SearchEntryPoint(
    rendered_content="""<style>
.container {
  align-items: center;
  border-radius: 8px;
  display: flex;
  font-family: Google 

Here is a plan for your trip to San Francisco, combining the research results:

*   **Museum:** San Francisco Museum of Modern Art (SFMOMA)
    *   This prominent museum is celebrated for its extensive collection of modern and contemporary art, featuring works by renowned artists such as Frida Kahlo and Andy Warhol. It spans seven gallery floors and showcases a commitment to both international and Bay Area artists.
*   **Concert:** Billy Idol with Joan Jett & The Blackhearts
    *   Catch the "It's a Nice Day To... Tour Again!" featuring Grammy-nominated Billy Idol and American rock band Joan Jett & The Blackhearts. This concert promises a blend of classic hits and new material.
*   **Restaurant:** House of Prime Rib
    *   A true San Francisco institution, House of Prime Rib has been serving 21-day aged prime rib since 1949. Every dinner includes a tossed salad, mashed or baked potato, and Yorkshire pudding, all enjoyed in a warm, English-style dining room with fireplaces.

--------------------------------------------------

--- parallel_planner_agent Complete ---
